# Setup

In [1]:
#***Importing libraries***

#Math tools
import numpy as np

#Data science tools
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

#OS tools
import os
import time

#Image processing tools
import cv2 

#Plotting tools
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
# Random seeds
np.random.seed(1)

# Custom Methods

In [3]:
### Error definition ###
def error_fn(y_true, y_pred):
    error = 0;
    for i in range(0,len(y_true)):
        if y_true[i] != y_pred[i]:
            error += 1; #Counts the number of misclassifications
    return error

# Reading the images

In [4]:
#Specifying the number of classes and images to use
n_classes = 2;
n_figs = 100;

#Specifying the total number of classes and images avaialble
N_classes = 45;
N_figs = 700;

#Randomly choosing the classes and figures to analyze
class_idx = np.random.permutation(N_classes)[:n_classes];
fig_idx = np.random.permutation(N_figs)[:n_figs]
#These two lines above generate different choices each time they're executed unless the random seed is reset

In [5]:
#Specifying the path
PROJECT_ROOT_DIR = '..'
DATASET_FOLDER = 'training_data'
DATASET_PATH = os.path.join(PROJECT_ROOT_DIR,DATASET_FOLDER)


#Reading the folders tree
classes_all = sorted(os.listdir(DATASET_PATH));
classes_all.remove('summary'); #Removes the summary file
N_classes = len(classes_all)
if n_classes > 0 and n_classes <= N_classes:
    classes = [classes_all[idx] for idx in class_idx]
else:
    print('Wrong number of classes requested')
    raise SystemExit(0)
    
#Reading the figures
figs = [];
for folder in classes:
    CLASS_PATH = os.path.join(DATASET_PATH, folder);
    figs_names_all = sorted(os.listdir(CLASS_PATH));
    if n_figs > 0 and n_figs <= N_figs:
        figs_names = [figs_names_all[idx] for idx in fig_idx]
        FIGS_PATHS = [os.path.join(CLASS_PATH, fig) for fig in figs_names]
        figs_i = [cv2.imread(PATH,0) for PATH in FIGS_PATHS]; #cv2.imread with flag 0 reads the image in grayscale
        figs.append(figs_i)
    else:
        print('Wrong number of figures requested')
        raise SystemExit(0)

#Converting into a numpy array
figs = np.array(figs);
print('figs.shape')
print(figs.shape)

##***Output***
# figs: Multidimensional numpy array containing all read images. First dimension corresponds to class and second dimension to figures. 
#Each figure is a multidimensional array itself of 256x256 pixels and only the greyscale channel: int(0,255)

figs.shape
(2, 100, 256, 256)


In [6]:
#Shows the names of the chosen classes and their corresponding indexes in figs
print('Chosen classes:')
print(list(enumerate(classes)))

Chosen classes:
[(0, 'basketball_court'), (1, 'baseball_diamond')]


In [7]:
#Visualizing some of the data
fig_test = figs[0][0];
plt.imshow(fig_test, cmap = mpl.cm.binary); plt.axis("off");

<IPython.core.display.Javascript object>

# Preparing the data set

In [8]:
#Linearizing the input data
X = figs.reshape([n_classes*n_figs, 256, 256]); #Inputs
y = np.zeros([n_classes, n_figs], dtype=int);
for i in range (n_classes):
    for j in range(n_figs):
        y[i,j] = i;
y = y.reshape([n_classes*n_figs]); #Labels

In [9]:
#Shuffling the input data
idx_random = np.random.permutation(len(X));
X = X[idx_random,:,:];
y = y[idx_random]

In [10]:
#Visualizing some of the data
idx = 11;
X_sample = X[idx];
plt.imshow(X_sample, cmap = mpl.cm.binary); plt.axis("off");
print(y[idx])
print(classes[y[idx]])

1
baseball_diamond


In [11]:
#Reshaping the dataset to a 2D array
X = X.reshape(-1, 256*256);
X.shape

(200, 65536)

In [12]:
#Splitting the data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, train_size=0.75, random_state=0) #Optimize for classification problems
N_train = len(X_train)
N_test = len(X_test)

In [13]:
X_train.shape

(150, 65536)

# Training the SVM classifier

In [14]:
#Setting up the models
C = 1  # SVM regularization parameter
models = (svm.SVC(kernel='linear', C=C),
          svm.SVC(kernel='poly', degree=2, gamma='auto', C=C),
          svm.SVC(kernel='poly', degree=3, gamma='auto', C=C),
          svm.SVC(kernel='rbf', gamma=0.7, C=C))
titles = ('SVM with linear kernel',
          'SVM with polynomial (degree 2) kernel',
          'SVM with polynomial (degree 3) kernel',
          'SVM with RBF kernel')
#Training 
models = (clf.fit(X_train, y_train) for clf in models)

## Training and generalization error

In [15]:
start_time = time.time()
i = 0;
error_train = np.zeros(len(titles))
error_test = np.zeros(len(titles))
print_results = True;
if print_results: print('*** Results ***')
for clf in models:       
    print(titles[i])
    #Train
    y_pred_train = clf.predict(X_train)
    error_train[i] = error_fn(y_train, y_pred_train)
    #Test
    y_pred_test = clf.predict(X_test)
    error_test[i] = error_fn(y_test, y_pred_test)
    if print_results:
        print('Train:'+str(y_pred_train))
        print('Train:'+str(y_pred_test)+'\n')
    i += 1

end_time = time.time()

if print_results:
    print('*** Truth ***')
    print('Train'+str(y_train))
    print('Test'+str(y_test))

print('\n*** Training error ***')
print('N_train: '+str(N_train))
i = 0
for title in titles:
    print(title+': '+str(error_train[i]))
    i += 1
print('\n*** Generalization error ***')
print('N_test: '+str(N_test))
i = 0
for title in titles:
    print(title+': '+str(error_test[i]))
    i += 1
    
print('\n Execution time %0.2f s'%(end_time-start_time))

*** Results ***
SVM with linear kernel
Train:[1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1
 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0
 0 0]
Train:[0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1
 1 0 0 1 1 1 0 1 0 0 0 0 1]

SVM with polynomial (degree 2) kernel
Train:[1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 0 0 1
 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0
 0 0]
Train:[0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 0 0 1
 1 1 0 1 1 1 0 1 0 0 0 0 1]

SVM with polynomial (degree 3) kernel
Train:[1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 1 

> ***The model is clearly overfitted!!!!***

# Best k-rank approximation through PCA

**Key questions**
- How to efficiently apply SVD to a batch of images and make decision about the number of modes to use? 
- Do we apply SVD to each input image, each class, or to the training dataset as a whole?
- What could be the optimum number of principal components to use? Which metric do we use to compare them?
- Training the model with the reduced components turned out to be much more computationall y expensive than training with the raw images. Why???

In [16]:
pca = PCA();
X_train_centered = X_train - X_train.mean(axis=0) # Test dataset
pca.fit(X_train_centered)
cumsum = np.cumsum(pca.explained_variance_ratio_)  #Note that the max number of singular values is the number of images in X_train
threshold = 0.95;
d = np.argmax(cumsum >= threshold) + 1

In [17]:
d

112

In [18]:
fig, subplt = plt.subplots(1, 2, figsize=(12, 6))

ax = subplt[0];
#ax.semilogy(range(1,len(cumsum)+1),(pca.singular_values_), 'o', linewidth=3)
ax.stem(range(1,len(cumsum)+1),(pca.singular_values_),basefmt=' ')
ax.set_xlabel("Dimensions", fontsize=15)
ax.set_ylabel("Singular values", fontsize=15)
ax.set_xlim([0,30])
ax.grid(True)


ax = subplt[1];
ax.plot(range(1,len(cumsum)+1),cumsum, linewidth=3)
ax.set_xlabel("Dimensions", fontsize=15)
ax.set_ylabel("Explained Variance", fontsize=15)

ax.plot([d, d], [0, threshold], "k:")
ax.plot([0, d], [threshold, threshold], "k:")
ax.plot(d, threshold, "ko")
ax.set_ylim([0,1.05])
ax.set_xlim([0,len(cumsum)+1])
ax.grid(True)



plt.show()

<IPython.core.display.Javascript object>

In [19]:
pca = PCA(n_components=0.8)
X_train_centered = X_train - X_train.mean(axis=0) # Test dataset
pca.fit(X_train_centered)
X_train_reduced = pca.fit_transform(X_train)
X_train_recovered = pca.inverse_transform(X_train_reduced)

In [20]:
pca.n_components_

59

In [21]:
np.sum(pca.explained_variance_ratio_)

0.8020027171336485

In [22]:
### Projecting the rest of the data onto the PCs of the training data ###
#Centers the data
X_centered = X - X.mean(axis=0) # Whole dataset
X_test_centered = X_test - X_test.mean(axis=0) # Test dataset
#Extract the normal vectors of the principal components
V_r = np.transpose(pca.components_); #pca.components = V^T
#Projects the data
X_reduced = np.matmul(X_centered, V_r);
X_test_reduced = np.matmul(X_test_centered, V_r)
X_test_recovered = pca.inverse_transform(X_test_reduced) #Equivalent to: np.matmul(X_test_reduced,np.transpose(V_r))

In [23]:
X_train_recovered.shape

(150, 65536)

In [24]:
#Visualizing some of the data - TRAINING
idx = 50;
#Train
X_sample = X_train[idx].reshape(256,256);
X_sample_recons = X_train_recovered[idx].reshape(256,256);
print(classes[y_train[idx]])
#Plotting
fig, sub = plt.subplots(1, 2, figsize=(8, 6))
sub[0].imshow(X_sample, cmap = mpl.cm.binary); sub[0].axis("off");
sub[1].imshow(X_sample_recons, cmap = mpl.cm.binary); sub[1].axis("off");

basketball_court


<IPython.core.display.Javascript object>

In [25]:
#Visualizing some of the data - TEST
idx = 11;
#Test
X_sample = X_test[idx].reshape(256,256);
X_sample_recons = X_test_recovered[idx].reshape(256,256);
print(classes[y_test[idx]])
#Plotting
fig, sub = plt.subplots(1, 2, figsize=(8, 6))
sub[0].imshow(X_sample, cmap = mpl.cm.binary); sub[0].axis("off");
sub[1].imshow(X_sample_recons, cmap = mpl.cm.binary); sub[1].axis("off");

baseball_diamond


<IPython.core.display.Javascript object>

In [26]:
#Vizualizing the data in based on the first principal component
X0 = X_reduced[:, 0];
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
for g in np.unique(y):
    idx = np.where(y == g);
    ax.scatter(X0[idx], y[idx], s=100, edgecolors='k', label=classes[g])
ax.set_xlabel('PC1', fontsize=15)
ax.set_ylabel('PC2', fontsize=15)
ax.set_xticks(())
ax.set_yticks(())
ax.legend(fontsize=13)
plt.show()

<IPython.core.display.Javascript object>

In [27]:
#Vizualizing the data in based on the first two principal components
X0, X1 = X_reduced[:, 0], X_reduced[:, 1];
fig, ax = plt.subplots(1, 1, figsize=(8, 6));
for g in np.unique(y):
    idx = np.where(y == g);
    ax.scatter(X0[idx], X1[idx], s=100, edgecolors='k', label=classes[g])
ax.set_xlabel('PC1', fontsize=15)
ax.set_ylabel('PC2', fontsize=15)
ax.set_xticks(())
ax.set_yticks(())
ax.legend(fontsize=13)
plt.show()

<IPython.core.display.Javascript object>

In [28]:
#Vizualizing the data in based on the first three principal components
X0, X1, X2 = X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2];
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
for g in np.unique(y):
    idx = np.where(y == g);
    ax.scatter(X0[idx], X1[idx], X2[idx], s=100, edgecolors='k', label=classes[g])
ax.set_xlabel('PC1', fontsize=15)
ax.set_ylabel('PC2', fontsize=15)
ax.set_zlabel('PC3', fontsize=15)
ax.set_xticks(())
ax.set_yticks(())
ax.set_zticks(())
ax.legend(fontsize=13)

#ax.view_init(80, angle)

plt.show()


<IPython.core.display.Javascript object>

In [29]:
y.shape

(200,)

# Training the SVM classifier on the reduced space

In [30]:
#Setting up the models
C = 1  # SVM regularization parameter
models = (svm.SVC(kernel='linear', C=C),
          svm.SVC(kernel='poly', degree=2, gamma='auto', C=C),
          svm.SVC(kernel='poly', degree=3, gamma='auto', C=C),
          svm.SVC(kernel='rbf', gamma=0.7, C=C))
titles = ('SVM with linear kernel',
          'SVM with polynomial (degree 2) kernel',
          'SVM with polynomial (degree 3) kernel',
          'SVM with RBF kernel')
#Training 
models = (clf.fit(X_train_reduced, y_train) for clf in models)

## Training and generalization error

In [ ]:
start_time = time.time()
i = 0;
error_train = np.zeros(len(titles))
error_test = np.zeros(len(titles))
print_results = True;
if print_results: print('*** Results ***')
for clf in models:       
    print(titles[i])
    #Train
    y_pred_train = clf.predict(X_train_reduced)
    error_train[i] = error_fn(y_train, y_pred_train)
    #Test
    y_pred_test = clf.predict(X_test_reduced)
    error_test[i] = error_fn(y_test, y_pred_test)
    if print_results:
        print('Train:'+str(y_pred_train))
        print('Test:'+str(y_pred_test)+'\n')
    i += 1

end_time = time.time()

if print_results:
    print('*** Truth ***')
    print('Train'+str(y_train))
    print('Test'+str(y_test))

print('\n*** Training error ***')
print('N_train: '+str(N_train))
i = 0
for title in titles:
    print(title+': '+str(error_train[i]))
    i += 1
print('\n*** Generalization error ***')
print('N_test: '+str(N_test))
i = 0
for title in titles:
    print(title+': '+str(error_test[i]))
    i += 1
    
print('\n Execution time %0.2f s'%(end_time-start_time))

*** Results ***


# Reduced input space through random projection (maybe?)

**Key questions**
- What kind of information/improvements would we get this way compared to PCA?

In [ ]:
<code>

# Cool ways to visualize the performance of the classifier

**Ideas**
- Manually add noise/perturbations to the images to test the classfier under extreme scenarios
- We could augment our training data set this way too! Do we need more data or are 700 images per class enough?
- Could we extende the trained model to be interactive such that we can upload a picture and it gets classified instantly?

In [ ]:
<code>